## Import libraries

In [11]:
import ee
import geemap
import geemap.chart as chart

## Create an interactive map

In [12]:
Map = geemap.Map(center=[40, -100], zoom=4)

## Add Earth Engine Python script

In [13]:
# Add Earth Engine dataset
image = ee.Image("USGS/SRTMGL1_003")

#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Chapter:      F2.2 Accuracy Assessment: Quantifying Classification Quality
#  Checkpoint:   F22b
#  Authors:      Andréa Puzzi Nicolau, Karen Dyson, David Saah, Nicholas Clinton
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# Import the reference dataset.
data = ee.FeatureCollection(
'projects/gee-book/assets/F2-2/milan_data')

# Define the prediction bands.
predictionBands = [
'SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7',
'ST_B10',
'ndvi', 'ndwi'
]

# Split the dataset into training and testing sets.
trainingTesting = data.randomColumn()
trainingSet = trainingTesting \
.filter(ee.Filter.lessThan('random', 0.8))
testingSet = trainingTesting \
.filter(ee.Filter.greaterThanOrEquals('random', 0.8))

# Train the Random Forest Classifier with the trainingSet.
RFclassifier = ee.Classifier.smileRandomForest(50).train(
features = trainingSet,
classProperty = 'class',
inputProperties = predictionBands
)

# Now, to test the classification (verify model's accuracy),
# we classify the testingSet and get a confusion matrix.
confusionMatrix = testingSet.classify(RFclassifier) \
.errorMatrix(
actual = 'class',
predicted = 'classification'
)

# Print the results.
print('Confusion matrix:', confusionMatrix.getInfo())
print('Overall Accuracy:', confusionMatrix.accuracy().getInfo())
print('Producers Accuracy:', confusionMatrix.producersAccuracy().getInfo())
print('Consumers Accuracy:', confusionMatrix.consumersAccuracy().getInfo())
print('Kappa:', confusionMatrix.kappa().getInfo())

#  -----------------------------------------------------------------------
#  CHECKPOINT
#  -----------------------------------------------------------------------

# Hyperparameter tuning.
numTrees = ee.List.sequence(5, 100, 5)


def func_tjn(t):
    classifier = ee.Classifier.smileRandomForest(t) \
    .train(
        features = trainingSet,
        classProperty = 'class',
        inputProperties = predictionBands
        )
    return testingSet \
    .classify(classifier) \
    .errorMatrix('class', 'classification') \
    .accuracy()

accuracies = numTrees.map(func_tjn)


chart = chart.array_values(
array = ee.Array(accuracies),
axis = 0,
x_labels = numTrees,
chart_type = "ScatterChart"
)

chart.set_options(options={
'hAxis': {
    'title': 'Number of trees'
},
'vAxis': {
    'title': 'Accuracy'
},
'title': 'Accuracy per number of trees'
})

chart.display()

#  -----------------------------------------------------------------------
#  CHECKPOINT
#  -----------------------------------------------------------------------

Confusion matrix: [[23, 0, 1, 0], [0, 21, 0, 4], [0, 1, 9, 1], [0, 6, 0, 12]]
Overall Accuracy: 0.8333333333333334
Producers Accuracy: [[0.9583333333333334], [0.84], [0.8181818181818182], [0.6666666666666666]]
Consumers Accuracy: [[1, 0.75, 0.9, 0.7058823529411765]]
Kappa: 0.7703804347826089


Figure(axes=[Axis(label_offset='30px', scale=LinearScale()), Axis(label_offset='40px', orientation='vertical',…

## Display the interactive map

In [ ]:
Map